# 📘 Expense Claim Automation — Microsoft Agent Framework

This notebook shows how to build a **tool-augmented AI agent** that reads expense data from a file and, upon user request, **uses a tool function** to (simulated) send an email to the finance team.

**What you will learn**
- How to define a *tool function* (here: `send_email`) with **Pydantic** parameter annotations
- How to create a **ChatAgent** using `AzureAIAgentClient` and `DefaultAzureCredential`
- How to load input data from a file and pass it to the agent
- How to run async agent calls safely in notebooks using `await`


## ✅ Prerequisites

1. An **Azure AI** project and a deployed chat model (e.g., Azure OpenAI).
2. These environment variables available to the notebook session (via shell or a `.env` file):
   - `PROJECT_ENDPOINT` → your Azure AI project endpoint
   - `MODEL_DEPLOYMENT_NAME` → your model deployment name (e.g., `gpt-4o-mini`)
3. A file named **`data.txt`** in the current working directory, containing itemized expenses (e.g., lines like `Taxi: 24.50`, `Hotel: 120.00`).

> Authentication: this notebook uses **`DefaultAzureCredential`**. For local dev, sign in with `az login`; in Azure, prefer Managed Identity.


# 📦 1) Import Required Libraries
If imports fail, ensure your environment has the required packages installed (see the optional next cell).

In [1]:
# Standard libs
import os
from dotenv import load_dotenv
from pathlib import Path
import asyncio

# Agent Framework / Azure identity
from agent_framework import ChatAgent, AgentThread  # AgentThread not used here; imported for reference
from agent_framework.azure import AzureAIAgentClient
from azure.identity import DefaultAzureCredential

# Tool parameter annotations
from pydantic import Field
from typing import Annotated


### (Optional) Install dependencies
Uncomment and run if you need to install packages in this environment.

In [2]:
# %pip install python-dotenv azure-identity pydantic
# %pip install agent-framework  # if not already available in your environment


# 🔧 2) Load Environment Variables & Validate Configuration
We load `.env` if present and **fail fast** if any required setting is missing.

In [3]:
load_dotenv()

def get_env(name: str) -> str:
    value = os.getenv(name)
    if not value:
        raise RuntimeError('Missing required env var: ' + name)
    return value

endpoint = get_env('PROJECT_ENDPOINT')
model = get_env('MODEL_DEPLOYMENT_NAME')
print('Environment OK')


Environment OK


# 🧰 3) Define the Email Tool Function (`send_email`)
This simulates sending an email by printing the message to stdout.

**Why Pydantic `Field`?** The annotations help the agent form well-structured tool calls with clear parameter semantics.

In [4]:
def send_email(
    to: Annotated[str, Field(description='whom to send the email to')],
    subject: Annotated[str, Field(description='subject of the email')],
    body: Annotated[str, Field(description='The text body of the email')]
):
    print('\nTo:', to)
    print('Subject:', subject)
    print(body, '\n')


# 🤖 4) Create the Expense Agent
We use `AzureAIAgentClient` with `DefaultAzureCredential`. The **instructions** make the tool usage explicit and deterministic to reduce output variance.

> **Note:** The tool is **registered** by passing the function to `tools=`. The agent can then call it when needed based on the instructions and user input.

In [5]:
credential = DefaultAzureCredential()
agent_name = 'expenses_agent'
instructions = (
    'You are an AI assistant for expense claim submission. '
    'When a user submits expenses data and requests an expense claim, use the plug-in function '
    "to send an email to expenses@contoso.com with the subject 'Expense Claim' and a body that "
    'contains itemized expenses with a total. Then confirm to the user that you have done so.'
)

agent = ChatAgent(
    chat_client=AzureAIAgentClient(
        credential=credential,
        project_endpoint=endpoint,
        model_deployment_name=model
    ),
    name=agent_name,
    instructions=instructions,
    tools=send_email
)
print('Agent ready:', agent_name)


Agent ready: expenses_agent


# 📄 5) Load Expense Data from File
Reads `data.txt` from the **current working directory**. Place your itemized entries there (one per line).

In [ ]:
curr_dir = os.getcwd()
data_path = Path(curr_dir) / 'data.txt'

if not data_path.exists():
    raise FileNotFoundError('data.txt not found in: ' + str(curr_dir))

with data_path.open('r', encoding='utf-8') as file:
    data = file.read() + '\n'
print('Loaded data from:', data_path)
print('Preview:\n' + data[:250])


# 🧮 6) Define the Processing Function
Wraps the **agent invocation** in an async function for reuse and clarity. It prints the agent's final response. The agent may call the **`send_email` tool** during execution, which will print the simulated email to stdout.

In [7]:
async def process_expenses_data(user_prompt: str, data: str, agent: ChatAgent):
    # Construct a simple message that pairs the instruction text with the data body
    prompt_message = [f'{user_prompt}: {data}']
    # Run the agent (awaitable). Depending on your SDK version, this returns the final text or an object.
    response = await agent.run(prompt_message)
    print('\n# Agent Response:\n' + str(response))


# ▶️ 7) Execute the Interaction (Notebook‑Safe Async)
In notebooks, do **not** call `asyncio.run()` (there is already an event loop). Use `await` directly. If running in a non-interactive environment, a fallback prompt is used.

In [8]:
try:
    user_prompt = input(
        f"Here is the expenses data in your file:\n\n{data}\n\nWhat would you like me to do with it?\n\n"
    )
    if not user_prompt.strip():
        user_prompt = 'Please submit an expense claim for these items'
except Exception:
    # Fallback for environments that do not support input()
    user_prompt = 'Please submit an expense claim for these items'

# Run the async function using notebook-safe await
await process_expenses_data(user_prompt, data, agent)



To: expenses@contoso.com
Subject: Expense Claim
Expense claim submission:

1. Date: 07-Mar-2025, Description: Taxi, Amount: $24.00
2. Date: 07-Mar-2025, Description: Dinner, Amount: $65.50
3. Date: 07-Mar-2025, Description: Hotel, Amount: $125.90

Total: $215.40 


# Agent Response:
I have submitted the expense claim to expenses@contoso.com, including your taxi, dinner, and hotel expenses totaling $215.40.


# 🧪 Troubleshooting & Tips
- **Auth errors**: run `az login` locally, or configure Managed Identity when running in Azure.
- **Missing env vars**: set `PROJECT_ENDPOINT` and `MODEL_DEPLOYMENT_NAME` or create a `.env` file.
- **No `data.txt`**: place a file named `data.txt` next to this notebook with itemized expenses.
- **Tool I/O**: `send_email` prints to stdout to simulate sending an email; replace with real mail API as needed.
- **Agent behavior**: keep instructions concise and deterministic. Use explicit rules and examples to reduce output drift.
